# HiPPO Matrices
---

## Table of Contents
* [Loading In Necessary Packages](#load-packages)
* [Instantiate The HiPPO Matrix](#instantiate-the-hippo-matrix)
    * [Translated Legendre (LegT)](#translated-legendre-legt)
        * [LegT](#legt)
        * [LMU](#lmu)
    * [Translated Laguerre (LagT)](#translated-laguerre-lagt)
    * [Scaled Legendre (LegS)](#scaled-legendre-legs)
    * [Fourier Basis](#fourier-basis)
        * [Fourier Recurrent Unit (FRU)](#fourier-recurrent-unit-fru)
        * [Truncated Fourier (FouT)](#truncated-fourier-fout)
        * [Fourier With Decay (FourD)](#fourier-with-decay-fourd)
* [Gu's Linear Time Invariant (LTI) HiPPO Operator](#gus-hippo-legt-operator)
* [Gu's Scale invariant (LSI) HiPPO Operator](#gus-scale-invariant-hippo-legs-operator)
* [Implementation Of General HiPPO Operator](#implementation-of-general-hippo-operator)
* [Test Generalized Bilinear Transform and Zero Order Hold Matrices](#test-generalized-bilinear-transform-and-zero-order-hold-matrices)
    * [Testing Forward Euler on GBT matrices](#testing-forward-euler-transform-for-lti-and-lsi)
    * [Testing Backward Euler on GBT matrices](#testing-backward-euler-transform-for-lti-and-lsi-on-legs-matrices)
    * [Testing Bidirectional on GBT matrices](#testing-lti-and-lsi-operators-with-bidirectional-transform)
    * [Testing ZOH on GBT matrices](#testing-zoh-transform-for-lti-and-lsi-on-legs-matrices)
* [Testing HiPPO Operators](#test-hippo-operators)
    * [Testing Forward Euler on HiPPO Operators](#testing-lti-and-lsi-operators-with-forward-euler-transform)
    * [Testing Backward Euler on HiPPO Operators](#testing-lti-and-lsi-operators-with-backward-euler-transform)
    * [Testing Bidirectional on HiPPO Operators](#testing-lti-and-lsi-operators-with-bidirectional-transform)
    * [Testing ZOH on HiPPO Operators](#testing-lti-and-lsi-operators-with-zoh-transform)
---


## Load Packages

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join("../../../"))
print(f"module_path: {module_path}")
if module_path not in sys.path:
    print(f"Adding {module_path} to sys.path")
    sys.path.append(module_path)

module_path: /home/beegass/Documents/Coding/HiPPO-Jax


In [2]:
# os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "False"
os.environ["TF_FORCE_UNIFIED_MEMORY"] = "1"

In [3]:
## import packages
import jax
from jax import numpy as jnp
import einops
import numpy as np
import torch

from src.models.hippo.hr_hippo import HRHiPPO_LSI, HRHiPPO_LTI
from src.models.hippo.hippo import HiPPOLTI, HiPPOLSI
from src.models.hippo.transition import TransMatrix
from src.data.process import whitesignal

/home/beegass/.cache/pypoetry/virtualenvs/hippo-pkg-Uqb72G6k-py3.8/lib/python3.8/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/home/beegass/.cache/pypoetry/virtualenvs/hippo-pkg-Uqb72G6k-py3.8/lib/python3.8/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


In [4]:
print(jax.devices())
print(f"The Device: {jax.lib.xla_bridge.get_backend().platform}")

[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]
The Device: gpu


In [5]:
print(f"MPS enabled: {torch.backends.mps.is_available()}")

MPS enabled: False


In [6]:
torch.set_printoptions(linewidth=150)
np.set_printoptions(linewidth=150)
jnp.set_printoptions(linewidth=150)

In [7]:
seed = 1701
key = jax.random.PRNGKey(seed)

In [8]:
num_copies = 10
subkeys = jax.random.split(key, num=num_copies)
key = subkeys[0]

## Test Generalized Bilinear Transform and Zero Order Hold Matrices

In [9]:
def test_LSI_GBT(hippo, hr_hippo, A, B, random_input, alpha=0.5, print_all=False):
    L = random_input.shape[1]

    flag = True
    for i in range(1, L + 1):
        GBT_A, GBT_B = hippo.discretize(A, B, step=i, alpha=alpha, dtype=jnp.float32)
        hr_GBT_A, hr_GBT_B = (
            jnp.asarray(hr_hippo.A_stacked[i - 1], dtype=jnp.float32),
            jnp.expand_dims(
                jnp.asarray(hr_hippo.B_stacked[i - 1], dtype=jnp.float32), axis=1
            ),
        )

        if print_all:
            print(f"GBT_A: {jnp.allclose(GBT_A, hr_GBT_A, rtol=1e-04, atol=1e-04)}")
            print(f"GBT_B: {jnp.allclose(GBT_B, hr_GBT_B, rtol=1e-04, atol=1e-04)}\n")

            print(f"hr_GBT_A shape:{hr_GBT_A.shape}\n")
            print(f"GBT_A shape: {GBT_A.shape}\n")
            print(f"hr_GBT_B shape: {hr_GBT_B.shape}\n")
            print(f"GBT_B shape: {GBT_B.shape}")

            print(f"hr_GBT_A:\n{hr_GBT_A}\n")
            print(f"GBT_A:\n{GBT_A}\n")
            print(f"hr_GBT_B:\n{hr_GBT_B}\n")
            print(f"GBT_B:\n{GBT_B}")

        check_A = jnp.allclose(GBT_A, hr_GBT_A, rtol=1e-04, atol=1e-04)
        check_B = jnp.allclose(GBT_B, hr_GBT_B, rtol=1e-04, atol=1e-04)

        if not check_A and not check_B:
            flag = False

    if not print_all:
        print(f"\n---------------------------------------------")
        print(f"---------- The Test Passed: {flag} ----------")
        print(f"---------------------------------------------\n")

In [10]:
def test_LTI_GBT(hippo, hr_hippo, A, B, random_input, step, alpha=0.5, print_all=False):
    L = random_input.shape[1]
    flag = True
    GBT_A, GBT_B = hippo.discretize(A, B, step=step, alpha=alpha, dtype=jnp.float32)
    hr_GBT_A, hr_GBT_B = (
        jnp.asarray(hr_hippo.dA, dtype=jnp.float32),
        jnp.expand_dims(jnp.asarray(hr_hippo.dB, dtype=jnp.float32), axis=1),
    )

    check_A = jnp.allclose(GBT_A, hr_GBT_A, rtol=1e-04, atol=1e-04)
    check_B = jnp.allclose(GBT_B, hr_GBT_B, rtol=1e-04, atol=1e-04)

    if not check_A and not check_B:
        flag = False

    if print_all:
        print(f"GBT_A: {jnp.allclose(GBT_A, hr_GBT_A, rtol=1e-04, atol=1e-04)}")
        print(f"GBT_B: {jnp.allclose(GBT_B, hr_GBT_B, rtol=1e-04, atol=1e-04)}\n")

        print(f"hr_GBT_A shape:{hr_GBT_A.shape}\n")
        print(f"GBT_A shape: {GBT_A.shape}\n")
        print(f"hr_GBT_B shape: {hr_GBT_B.shape}\n")
        print(f"GBT_B shape: {GBT_B.shape}")

        print(f"hr_GBT_A:\n{hr_GBT_A}\n")
        print(f"GBT_A:\n{GBT_A}\n")
        print(f"hr_GBT_B:\n{hr_GBT_B}\n")
        print(f"GBT_B:\n{GBT_B}")

    if not print_all:
        print(f"\n---------------------------------------------")
        print(f"---------- The Test Passed: {flag} ----------")
        print(f"---------------------------------------------\n")

In [11]:
def test_GBT(
    the_measure="legs", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=False
):

    T = 3
    freq = 3
    step = 1e-3
    L = int(T / step)

    batch_size = 2
    data_size = L
    input_size = 1

    N = 10

    print(f"step: {step}")

    u = whitesignal(subkeys[4], T, step, freq, batch_shape=(batch_size,))

    x_np = np.asarray(u)

    x_np = einops.rearrange(x_np, "batch seq_len -> batch seq_len 1")

    # ----------------------------------------------------------------------------------
    # ------------------------------ Instantiate Gu's HiPPOs ---------------------------
    # ----------------------------------------------------------------------------------

    print(f"Creating Gu's HiPPO-{the_measure} LTI model with {alpha} transform")
    hr_hippo_lti = HRHiPPO_LTI(
        N=N,
        method=the_measure,
        dt=step,
        T=T,
        discretization=discretization,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        c=0.0,
    )  # The Gu's

    if the_measure == "legs":
        print(f"Creating Gu's HiPPO-{the_measure} LSI model with {alpha} transform")
        hr_hippo_lsi = HRHiPPO_LSI(
            N=N,
            method=the_measure,
            max_length=L,
            discretization=discretization,
            lambda_n=lambda_n,
            alpha=0.0,
            beta=1.0,
        )  # The Gu's

    matrices = TransMatrix(
        N=N,
        measure=the_measure,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        dtype=jnp.float32,
    )

    A = matrices.A
    B = matrices.B

    print(f"Creating HiPPO-{the_measure} LTI model with {alpha} transform")
    hippo_lti = HiPPOLTI(
        N=N,
        step_size=step,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        GBT_alpha=alpha,
        measure=the_measure,
        basis_size=T,
        dtype=jnp.float32,
        unroll=False,
    )  # Bryan's

    lti_variables = hippo_lti.init(subkeys[2], x_np)
    hippo_lti = hippo_lti.bind(lti_variables)

    if the_measure == "legs":
        print(f"Creating HiPPO-{the_measure} LSI model with {alpha} transform")
        hippo_lsi = HiPPOLSI(
            N=N,
            max_length=L,
            step_size=step,
            lambda_n=lambda_n,
            alpha=0.0,
            beta=1.0,
            GBT_alpha=alpha,
            measure=the_measure,
            dtype=jnp.float32,
            unroll=True,
        )  # Bryan's

        lsi_variables = hippo_lsi.init(subkeys[3], x_np)
        hippo_lsi = hippo_lsi.bind(lsi_variables)

    print(f"Testing for correct LTI GBT matrices for HiPPO-{the_measure}")
    test_LTI_GBT(
        hippo=hippo_lti,
        hr_hippo=hr_hippo_lti,
        A=A,
        B=B,
        random_input=x_np,
        step=step,
        alpha=alpha,
        print_all=print_all,
    )
    if the_measure == "legs":
        print(f"Testing for correct LSI GBT matrices for HiPPO-{the_measure}")
        test_LSI_GBT(
            hippo=hippo_lsi,
            hr_hippo=hr_hippo_lsi,
            A=A,
            B=B,
            random_input=x_np,
            alpha=alpha,
            print_all=print_all,
        )

## Navigation To Table Of Contents
---
* [Table Of Contents](#table-of-contents)
* [Loading In Necessary Packages](#load-packages)
* [Instantiate The HiPPO Matrix](#instantiate-the-hippo-matrix)
* [Gu's Linear Time Invariant (LTI) HiPPO Operator](#gus-hippo-legt-operator)
* [Gu's Scale invariant (LSI) HiPPO Operator](#gus-scale-invariant-hippo-legs-operator)
* [Implementation Of General HiPPO Operator](#implementation-of-general-hippo-operator)
* [Test Generalized Bilinear Transform and Zero Order Hold Matrices](#test-generalized-bilinear-transform-and-zero-order-hold-matrices)
* [Testing HiPPO Operators](#test-hippo-operators)
---

In [12]:
print_all = False

### Testing Forward Euler Transform for LTI and LSI

#### LegS

In [13]:
test_GBT(
    the_measure="legs", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-legs LTI model with 0.0 transform
Creating Gu's HiPPO-legs LSI model with 0.0 transform
Creating HiPPO-legs LTI model with 0.0 transform
Creating HiPPO-legs LSI model with 0.0 transform
Testing for correct LTI GBT matrices for HiPPO-legs

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------

Testing for correct LSI GBT matrices for HiPPO-legs

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### LegT

In [14]:
test_GBT(
    the_measure="legt", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-legt LTI model with 0.0 transform
Creating HiPPO-legt LTI model with 0.0 transform
Testing for correct LTI GBT matrices for HiPPO-legt

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### LMU

In [15]:
test_GBT(
    the_measure="lmu", lambda_n=2.0, alpha=0.0, discretization=0.0, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-lmu LTI model with 0.0 transform
Creating HiPPO-lmu LTI model with 0.0 transform
Testing for correct LTI GBT matrices for HiPPO-lmu

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### LagT

In [16]:
test_GBT(
    the_measure="lagt", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-lagt LTI model with 0.0 transform
Creating HiPPO-lagt LTI model with 0.0 transform
Testing for correct LTI GBT matrices for HiPPO-lagt

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### FRU

In [17]:
test_GBT(
    the_measure="fru", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-fru LTI model with 0.0 transform
Creating HiPPO-fru LTI model with 0.0 transform
Testing for correct LTI GBT matrices for HiPPO-fru

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### FouT

In [18]:
test_GBT(
    the_measure="fout", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-fout LTI model with 0.0 transform
Creating HiPPO-fout LTI model with 0.0 transform
Testing for correct LTI GBT matrices for HiPPO-fout

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### FouD

In [19]:
test_GBT(
    the_measure="foud", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-foud LTI model with 0.0 transform
Creating HiPPO-foud LTI model with 0.0 transform
Testing for correct LTI GBT matrices for HiPPO-foud

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



### Testing Backward Euler Transform for LTI and LSI on LegS Matrices

#### LegS

In [20]:
test_GBT(
    the_measure="legs", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-legs LTI model with 1.0 transform
Creating Gu's HiPPO-legs LSI model with 1.0 transform
Creating HiPPO-legs LTI model with 1.0 transform
Creating HiPPO-legs LSI model with 1.0 transform
Testing for correct LTI GBT matrices for HiPPO-legs

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------

Testing for correct LSI GBT matrices for HiPPO-legs

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### LegT

In [21]:
test_GBT(
    the_measure="legt", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-legt LTI model with 1.0 transform
Creating HiPPO-legt LTI model with 1.0 transform
Testing for correct LTI GBT matrices for HiPPO-legt

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### LMU

In [22]:
test_GBT(
    the_measure="lmu", lambda_n=2.0, alpha=1.0, discretization=1.0, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-lmu LTI model with 1.0 transform
Creating HiPPO-lmu LTI model with 1.0 transform
Testing for correct LTI GBT matrices for HiPPO-lmu

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### LagT

In [23]:
test_GBT(
    the_measure="lagt", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-lagt LTI model with 1.0 transform
Creating HiPPO-lagt LTI model with 1.0 transform
Testing for correct LTI GBT matrices for HiPPO-lagt

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### FRU

In [24]:
test_GBT(
    the_measure="fru", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-fru LTI model with 1.0 transform
Creating HiPPO-fru LTI model with 1.0 transform
Testing for correct LTI GBT matrices for HiPPO-fru

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### FouT

In [25]:
test_GBT(
    the_measure="fout", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-fout LTI model with 1.0 transform
Creating HiPPO-fout LTI model with 1.0 transform
Testing for correct LTI GBT matrices for HiPPO-fout

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### FouD

In [26]:
test_GBT(
    the_measure="foud", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-foud LTI model with 1.0 transform
Creating HiPPO-foud LTI model with 1.0 transform
Testing for correct LTI GBT matrices for HiPPO-foud

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



### Testing Bidirectional Transform for LTI and LSI on LegS Matrices

#### LegS

In [27]:
test_GBT(
    the_measure="legs", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-legs LTI model with 0.5 transform
Creating Gu's HiPPO-legs LSI model with 0.5 transform
Creating HiPPO-legs LTI model with 0.5 transform
Creating HiPPO-legs LSI model with 0.5 transform
Testing for correct LTI GBT matrices for HiPPO-legs

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------

Testing for correct LSI GBT matrices for HiPPO-legs

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### LegT

In [28]:
test_GBT(
    the_measure="legt", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-legt LTI model with 0.5 transform
Creating HiPPO-legt LTI model with 0.5 transform
Testing for correct LTI GBT matrices for HiPPO-legt

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### LMU

In [29]:
test_GBT(
    the_measure="lmu", lambda_n=2.0, alpha=0.5, discretization=0.5, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-lmu LTI model with 0.5 transform
Creating HiPPO-lmu LTI model with 0.5 transform
Testing for correct LTI GBT matrices for HiPPO-lmu

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### LagT

In [30]:
test_GBT(
    the_measure="lagt", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-lagt LTI model with 0.5 transform
Creating HiPPO-lagt LTI model with 0.5 transform
Testing for correct LTI GBT matrices for HiPPO-lagt

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### FRU

In [31]:
test_GBT(
    the_measure="fru", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-fru LTI model with 0.5 transform
Creating HiPPO-fru LTI model with 0.5 transform
Testing for correct LTI GBT matrices for HiPPO-fru

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### FouT

In [32]:
test_GBT(
    the_measure="fout", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-fout LTI model with 0.5 transform
Creating HiPPO-fout LTI model with 0.5 transform
Testing for correct LTI GBT matrices for HiPPO-fout

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### FouD

In [33]:
test_GBT(
    the_measure="foud", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

step: 0.001
Creating Gu's HiPPO-foud LTI model with 0.5 transform
Creating HiPPO-foud LTI model with 0.5 transform
Testing for correct LTI GBT matrices for HiPPO-foud

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



### Testing ZOH Transform for LTI and LSI on LegS Matrices

#### LegS

In [34]:
test_GBT(
    the_measure="legs",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

step: 0.001
Creating Gu's HiPPO-legs LTI model with 2.0 transform
Creating Gu's HiPPO-legs LSI model with 2.0 transform
Creating HiPPO-legs LTI model with 2.0 transform
Creating HiPPO-legs LSI model with 2.0 transform
Testing for correct LTI GBT matrices for HiPPO-legs

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------

Testing for correct LSI GBT matrices for HiPPO-legs

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### LegT

In [35]:
test_GBT(
    the_measure="legt",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

step: 0.001
Creating Gu's HiPPO-legt LTI model with 2.0 transform
Creating HiPPO-legt LTI model with 2.0 transform
Testing for correct LTI GBT matrices for HiPPO-legt

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### LMU

In [36]:
test_GBT(
    the_measure="lmu",
    lambda_n=2.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

step: 0.001
Creating Gu's HiPPO-lmu LTI model with 2.0 transform
Creating HiPPO-lmu LTI model with 2.0 transform
Testing for correct LTI GBT matrices for HiPPO-lmu

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### LagT

In [37]:
test_GBT(
    the_measure="lagt",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

step: 0.001
Creating Gu's HiPPO-lagt LTI model with 2.0 transform
Creating HiPPO-lagt LTI model with 2.0 transform
Testing for correct LTI GBT matrices for HiPPO-lagt

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### FRU

In [38]:
test_GBT(
    the_measure="fru",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

step: 0.001
Creating Gu's HiPPO-fru LTI model with 2.0 transform
Creating HiPPO-fru LTI model with 2.0 transform
Testing for correct LTI GBT matrices for HiPPO-fru

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### FouT

In [39]:
test_GBT(
    the_measure="fout",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

step: 0.001
Creating Gu's HiPPO-fout LTI model with 2.0 transform
Creating HiPPO-fout LTI model with 2.0 transform
Testing for correct LTI GBT matrices for HiPPO-fout

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------



#### FouD

In [40]:
test_GBT(
    the_measure="foud",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

step: 0.001
Creating Gu's HiPPO-foud LTI model with 2.0 transform
Creating HiPPO-foud LTI model with 2.0 transform
Testing for correct LTI GBT matrices for HiPPO-foud

---------------------------------------------
---------- The Test Passed: True ----------
---------------------------------------------

